# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

In [1]:
!pip install datasets
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset

dataset = load_dataset("emotion", trust_remote_code=True)
print(dataset)
train_dataset = dataset['train']

shuffled_train_dataset = train_dataset.shuffle(seed=42) 

# taking a smaller portion of the big dataset
fraction = 1/8
num_samples = int(len(shuffled_train_dataset) * fraction)

# Select the subset
subset_train_dataset = shuffled_train_dataset.select(range(num_samples))

train_dataset = subset_train_dataset
test_dataset = dataset['test']

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_batch(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

tokenized_train = train_dataset.map(tokenize_batch, batched=True)
tokenized_test = test_dataset.map(tokenize_batch, batched=True)
tokenized_train


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2000
})

In [3]:
from transformers import AutoModelForSequenceClassification

pre_trained_model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels = 6,
    id2label = {0:"sadness", 1:"joy", 2:"love", 3:"anger", 4:"fear", 5:"surprise"},
    label2id = {"sadness":0, "joy":1, "love":2, "anger":3, "fear":4, "surprise":5}
    )

pre_trained_model.config.pad_token_id = tokenizer.pad_token_id

for param in pre_trained_model.parameters():
    param.requires_grad = True
print(pre_trained_model)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)


In [4]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments 
import numpy as np
def compute_metrics(eval_pred):
    pred, label = eval_pred
    pred = np.argmax(pred, axis = 1)
    return {"accuracy": (pred == label).mean()}
    
trainer = Trainer(
    model=pre_trained_model,
    args= TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
   
),
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer = tokenizer)
)

trainer.train()
base_Evaluation = trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.474583,0.460000


Checkpoint destination directory ./results/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [5]:
print(base_Evaluation)

{'eval_loss': 1.4745830297470093, 'eval_accuracy': 0.46, 'eval_runtime': 745.1787, 'eval_samples_per_second': 2.684, 'eval_steps_per_second': 0.168, 'epoch': 1.0}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:
from peft import LoraConfig, TaskType
from transformers import AutoModelForCausalLM
from peft import get_peft_model
from transformers import GPT2TokenizerFast


config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
)

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2", 
    num_labels = 6,
    id2label = {0:"sadness", 1:"joy", 2:"love", 3:"anger", 4:"fear", 5:"surprise"},
    label2id = {"sadness":0, "joy":1, "love":2, "anger":3, "fear":4, "surprise":5}
    )
model.config.pad_token_id = model.config.eos_token_id

for param in model.parameters():
    param.requires_grad = True
    
lora_model = get_peft_model(model, config)
lora_model

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()

In [7]:
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
print("Sample from tokenized_train:", tokenized_train[0])

Sample from tokenized_train: {'label': tensor(4), 'input_ids': tensor([ 4514, 16259,   287,   262,  1499, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256]), 'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}


In [9]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments 
import numpy as np
def compute_metrics(eval_pred):
    pred, label = eval_pred
    pred = np.argmax(pred, axis = 1)
    return {"accuracy": (pred == label).mean()}





trainer = Trainer(
    model=lora_model,
    args= TrainingArguments(
    output_dir="./results_fineTuned",
    learning_rate=3e-5,
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
   
),
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer = tokenizer)
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,4.270235,0.172000


Checkpoint destination directory ./results_fineTuned/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=125, training_loss=4.6696181640625, metrics={'train_runtime': 2632.4421, 'train_samples_per_second': 0.76, 'train_steps_per_second': 0.047, 'total_flos': 65556578304000.0, 'train_loss': 4.6696181640625, 'epoch': 1.0})

In [10]:
lora_model.save_pretrained("models/gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [11]:
peft_model = AutoModelForSequenceClassification.from_pretrained(
    "models/gpt-lora",
    num_labels = 6,
    id2label = {0:"sadness", 1:"joy", 2:"love", 3:"anger", 4:"fear", 5:"surprise"},
    label2id = {"sadness":0, "joy":1, "love":2, "anger":3, "fear":4, "surprise":5}
    )

peft_model.config.pad_token_id = tokenizer.pad_token_id
peft_model

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear(
            in_features=768, out_features=2304, bias=True
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=768, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2304, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [12]:
trainer = Trainer(
    model=lora_model,
    args= TrainingArguments(
    output_dir="./results_fineTuned",
    learning_rate=2e-5,
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
   
),
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer = tokenizer)
)

fineTuned_Evaluation = trainer.evaluate()
print(fineTuned_Evaluation)

{'eval_loss': 4.27023458480835, 'eval_accuracy': 0.172, 'eval_runtime': 695.4233, 'eval_samples_per_second': 2.876, 'eval_steps_per_second': 0.18}


In [16]:
print("Fine Tuned Model Evaluation vs Base Model Evaluation")
for key in fineTuned_Evaluation.keys():
    print(f"{key}: {fineTuned_Evaluation[key]} | {base_Evaluation[key]}")

Fine Tuned Model Evaluation vs Base Model Evaluation
eval_loss: 4.27023458480835 | 1.4745830297470093
eval_accuracy: 0.172 | 0.46
eval_runtime: 695.4233 | 745.1787
eval_samples_per_second: 2.876 | 2.684
eval_steps_per_second: 0.18 | 0.168


In [1]:
!zip -r workspace.zip ./*

/usr/bin/sh: 1: zip: not found
